![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_CamemBertForTokenClassification.ipynb)

## Import ONNX CamemBertForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `CamemBertForTokenClassification` is only available since in `Spark NLP 5.2.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import CamemBERT models trained/fine-tuned for token classification via `CamembertForTokenClassification` or `TFCamembertForTokenClassification`. These models are usually under `Token Classification` category and have `camembert` in their labels
- Reference: [TFCamembertForTokenClassification](https://huggingface.co/docs/transformers/model_doc/camembert#transformers.TFCamembertForTokenClassification)
- Some [example models](https://huggingface.co/models?other=camembert&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.51.3`. This doesn't mean it won't work with the future releases
- CamembertTokenizer requires the `SentencePiece` library, so we install that as well

In [1]:
!pip install -q transformers[onnx]==4.51.3 optimum onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [Jean-Baptiste/camembert-ner](https://huggingface.co/Jean-Baptiste/camembert-ner) model from HuggingFace as an example
- In addition to the CamemBERT model, we also need to save the `CamembertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import CamembertTokenizer
from optimum.onnxruntime import ORTModelForTokenClassification

MODEL_NAME = 'Jean-Baptiste/camembert-ner'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL)

tokenizer = CamembertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(ONNX_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The model Jean-Baptiste/camembert-ner was already converted to ONNX but got `export=True`, the model will be converted to ONNX once again. Don't forget to save the resulting model with `.save_pretrained()`


config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

('onnx_models/Jean-Baptiste/camembert-ner/tokenizer_config.json',
 'onnx_models/Jean-Baptiste/camembert-ner/special_tokens_map.json',
 'onnx_models/Jean-Baptiste/camembert-ner/sentencepiece.bpe.model',
 'onnx_models/Jean-Baptiste/camembert-ner/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {ONNX_MODEL}

total 430948
-rw-r--r-- 1 root root        22 Jun 11 21:33 added_tokens.json
-rw-r--r-- 1 root root       886 Jun 11 21:33 config.json
-rw-r--r-- 1 root root 440459690 Jun 11 21:33 model.onnx
-rw-r--r-- 1 root root    810912 Jun 11 21:33 sentencepiece.bpe.model
-rw-r--r-- 1 root root       354 Jun 11 21:33 special_tokens_map.json
-rw-r--r-- 1 root root      1794 Jun 11 21:33 tokenizer_config.json


- We need to move `spiece.model` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [4]:
!mkdir -p {ONNX_MODEL}/assets

with open(f"{ONNX_MODEL}/assets/labels.txt", "w") as f:
    f.write("\n".join(sorted(ort_model.config.label2id, key=ort_model.config.label2id.get)))

!mv {ONNX_MODEL}/sentencepiece.bpe.model {ONNX_MODEL}/assets/

In [5]:
!ls -lR {ONNX_MODEL}

onnx_models/Jean-Baptiste/camembert-ner:
total 430160
-rw-r--r-- 1 root root        22 Jun 11 21:33 added_tokens.json
drwxr-xr-x 2 root root      4096 Jun 11 21:33 assets
-rw-r--r-- 1 root root       886 Jun 11 21:33 config.json
-rw-r--r-- 1 root root 440459690 Jun 11 21:33 model.onnx
-rw-r--r-- 1 root root       354 Jun 11 21:33 special_tokens_map.json
-rw-r--r-- 1 root root      1794 Jun 11 21:33 tokenizer_config.json

onnx_models/Jean-Baptiste/camembert-ner/assets:
total 796
-rw-r--r-- 1 root root     26 Jun 11 21:33 labels.txt
-rw-r--r-- 1 root root 810912 Jun 11 21:33 sentencepiece.bpe.model


In [6]:
!cat {ONNX_MODEL}/assets/labels.txt

O
I-LOC
I-PER
I-MISC
I-ORG

Voila! We have our `sentencepiece.bpe.model` and `labels.txt` inside assets directory

## Import and Save CamemBertForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab.
- For this example we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly.

In [7]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 52.7 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `CamemBertForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `CamemBertForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [9]:
from sparknlp.annotator import CamemBertForTokenClassification

tokenClassifier = CamemBertForTokenClassification.loadSavedModel(
        ONNX_MODEL,
        spark
    ) \
    .setInputCols(["document", "token"]) \
    .setOutputCol("ner") \
    .setCaseSensitive(True) \
    .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your CamemBertForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [12]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 431008
-rw-r--r-- 1 root root 440527034 Jun 11 21:37 camembert_classification_onnx
-rw-r--r-- 1 root root    810912 Jun 11 21:37 camembert_spp
drwxr-xr-x 3 root root      4096 Jun 11 21:37 fields
drwxr-xr-x 2 root root      4096 Jun 11 21:37 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny CamemBertForTokenClassification model 😊

In [13]:
tokenClassifier_loaded = CamemBertForTokenClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
    .setInputCols(["document",'token'])\
    .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [14]:
tokenClassifier_loaded.getClasses()

['I-ORG', 'I-MISC', 'I-LOC', 'I-PER', 'O']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [15]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, NerConverter

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

converter = NerConverter() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded,
    converter
])

example = spark.createDataFrame([
    ["Je m'appelle jean-baptiste et je vis à montréal"],
    ["george washington est allé à washington"]
], ["text"])

result = pipeline.fit(example).transform(example)
result.select("text", "ner.result").show(truncate=False)

result.selectExpr("explode(ner_chunk) as chunk").selectExpr(
    "chunk.result as text",
    "chunk.metadata['entity'] as entity"
).show(truncate=False)

+-----------------------------------------------+--------------------------------+
|text                                           |result                          |
+-----------------------------------------------+--------------------------------+
|Je m'appelle jean-baptiste et je vis à montréal|[O, O, I-PER, O, O, O, O, I-LOC]|
|george washington est allé à washington        |[I-PER, I-PER, O, O, O, I-LOC]  |
+-----------------------------------------------+--------------------------------+

+-----------------+------+
|text             |entity|
+-----------------+------+
|jean-baptiste    |PER   |
|montréal         |LOC   |
|george washington|PER   |
|washington       |LOC   |
+-----------------+------+



That's it! You can now go wild and use hundreds of `CamemBertForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
